In [4]:
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
from transformers import Trainer, TrainingArguments
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import f1_score, classification_report
import torch
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import numpy as np
from transformers import AdamW
from tqdm import tqdm

In [5]:
language="tr"
modelname="DistilBert"
input = "gpt4"
test = "Dodiom"
filtered= False

In [6]:
if input == "gpt4":
  gpt4_flag=True
else:
  gpt4_flag=False

In [7]:
# JP encoding='euc-jp' other encoding normal CHECK ALWAYS DATASETS
df_train = pd.read_csv(f'train_gpt4_tr_zero_shot.csv')
df_eval = pd.read_csv(f'eval_gpt4_tr_zero_shot.csv')
df_test = pd.read_csv(f'dodiom_tr_test.csv')

In [8]:
def get_category_mapping(language):
    if language in ['en', 'jp']:
        category_mapping = {
            'figurative': 'idiom',
            'literal': 'nonidiom'
        }
    elif language == 'tr':
        category_mapping = {
            'mecaz': 'idiom',
            'gerçek': 'nonidiom'
        }
    elif language == 'it':
        category_mapping = {
            'figurato': 'idiom',
            'letterale': 'nonidiom'
        }
    else:
        raise ValueError(f"Language '{language}' is not supported.")

    return category_mapping

In [9]:
category_mapping = get_category_mapping(language)
print(category_mapping)

{'mecaz': 'idiom', 'gerçek': 'nonidiom'}


In [10]:
if gpt4_flag or language in ['en', 'jp']:
  df_train['category'] = df_train['category'].map(category_mapping)
df_train

,submission,category,idiom,type
0,Eşitlik için yapılan mücadelelere karşı çıkmak...,idiom,Karşı çıkmak,Zero-shot
1,"Oyunun kırılma anı, takımının şampiyonluğa yol...",idiom,Yol açmak,Zero-shot
2,Komşuluk festivali düzenleme görevini üstlenen...,idiom,Kolları sıvamak,Zero-shot
3,Şirketteki başarısızlıkla ilgili yapılan yorum...,idiom,Üstüne almak,Zero-shot
4,Sorunları erkenden çözerek kriz anlarında zama...,idiom,Zaman kazanmak,Zero-shot
...,...,...,...,...
3451,Kütüphanede çalışmak için sessiz bir yer almış...,nonidiom,Yer almak,Zero-shot
3452,"Sprinter, yarışın son metresinde zamanı iyi ku...",nonidiom,Zaman kazanmak,Zero-shot
3453,Sergi alanının ortasına bir sanat eseri koydu ...,nonidiom,Ortaya koymak,Zero-shot
3454,Eski battaniyeyi köpeğin üstünden atmak için n...,nonidiom,Üstünden atmak,Zero-shot


In [11]:
if gpt4_flag or language in ['en', 'jp']:
  df_eval['category'] = df_eval['category'].map(category_mapping)
df_eval

,submission,category,idiom,type
0,Yeni öğrendiği şarkıyı sokak köşesinde söyleye...,idiom,İçi erimek,Zero-shot
1,Bir lider olarak çalışanlarının hatalarını def...,idiom,Defterden silmek,Zero-shot
2,"Yönetici, iş yerindeki moral ve motivasyonu yü...",idiom,Nabzını tutmak,Zero-shot
3,Şirketteki gizli toplantıyı kazara sızdırınca ...,idiom,Ayvayı yemek,Zero-shot
4,"Film eleştirmenleri, yönetmenin eserindeki sim...",idiom,Üzerinde durmak,Zero-shot
...,...,...,...,...
859,"Koşu parkurunda, atletler yorulan arkadaşların...",nonidiom,Başa geçmek,Zero-shot
860,"Kitapçı, yeni gelen kitapları rafa koymadan ön...",nonidiom,El koymak,Zero-shot
861,"Çocuklar, köpeklerini sıkı bir şekilde tutarak...",nonidiom,Sıkı durmak,Zero-shot
862,"Tiyatro oyuncuları, sahnenin üstüne çıkmak içi...",nonidiom,Üste çıkmak,Zero-shot


In [12]:
if language in ['en', 'jp']:
  df_test['category'] = df_test['category'].map(category_mapping)

In [13]:
df_test

,idiom,submission,category,rating,likes,dislikes,reports,idiom_indices,idiom_words,lemmas,words
0,adım atmak,"Sen bu adama adım mı attın ki, o sana yakınlaş...",idiom,0.75,3,0,1,"[3, 5]","['adım', 'attın']","['se', 'bu', 'ada', 'adım', 'mı', 'atmak', 'ki...","['Sen', 'bu', 'adama', 'adım', 'mı', 'attın', ..."
1,adım atmak,Bütün zorluklara rağmen yeni bir işe adım attı.,idiom,1.00,3,0,0,"[6, 7]","['adım', 'attı']","['bütün', 'zor', 'rağmen', 'yen', 'bir', 'işem...","['Bütün', 'zorluklara', 'rağmen', 'yeni', 'bir..."
2,adım atmak,Adım attığı bu hayatın başına ne gibi olaylar ...,idiom,0.75,3,0,1,"[0, 1]","['Adım', 'attığı']","['adım', 'atmak', 'bu', 'hayat', 'baş', 'ne', ...","['Adım', 'attığı', 'bu', 'hayatın', 'başına', ..."
3,adım atmak,"Bölüm başkanı, akademik dünyaya adım atan genç...",idiom,0.50,1,0,1,"[5, 6]","['adım', 'atan']","['bölüm', 'başka', ',', 'akademik', 'Dünya', '...","['Bölüm', 'başkanı', ',', 'akademik', 'dünyaya..."
4,adım atmak,Yeni bir deneyime adım atıyorum.,idiom,0.75,3,0,1,"[3, 4]","['adım', 'atıyorum']","['yen', 'bir', 'deneyim', 'adım', 'atmak', '.']","['Yeni', 'bir', 'deneyime', 'adım', 'atıyorum'..."
...,...,...,...,...,...,...,...,...,...,...,...
2079,üzerinde durmak,Bu ipin üzerinde duramıyorum.,nonidiom,1.00,5,0,0,"[2, 3]","['üzerinde', 'duramıyorum']","['bu', 'ip', 'Üzer', 'durmak', '.']","['Bu', 'ipin', 'üzerinde', 'duramıyorum', '.']"
2080,üzerinde durmak,İpin üzerinde tek ayağıyla duruyordu.,nonidiom,1.00,5,0,0,"[1, 4]","['üzerinde', 'duruyordu']","['ip', 'Üzer', 'tek', 'Ayağ', 'durmak', '.']","['İpin', 'üzerinde', 'tek', 'ayağıyla', 'duruy..."
2081,üzerinde durmak,Bilgisayarın üzerinde kitapları duruyor,nonidiom,1.00,4,0,0,"[1, 3]","['üzerinde', 'duruyor']","['bilgisayar', 'Üzer', 'kitap', 'durmak']","['Bilgisayarın', 'üzerinde', 'kitapları', 'dur..."
2082,üzerinde durmak,Üzerinde çilekler duran tartı bana uzattı.,nonidiom,1.00,1,0,0,"[0, 2]","['Üzerinde', 'duran']","['Üzer', 'Çilek', 'durmak', 'tartı', 'ban', 'u...","['Üzerinde', 'çilekler', 'duran', 'tartı', 'ba..."


In [14]:
if Filtered == True:
  df_train = df_train.groupby(['idiom', 'category']).apply(lambda x: x.sample(n=32, replace=False)).reset_index(drop=True)
  df_eval = df_eval.groupby(['idiom', 'category']).apply(lambda x: x.sample(n=8, replace=False)).reset_index(drop=True)

NameError: name 'Filtered' is not defined

In [ ]:
df_train['category'] = df_train['category'].str.lower()
df_eval['category'] = df_eval['category'].str.lower()
df_test['category'] = df_test['category'].str.lower()

In [15]:
df_train.head()

,submission,category,idiom,type
0,Eşitlik için yapılan mücadelelere karşı çıkmak...,idiom,Karşı çıkmak,Zero-shot
1,"Oyunun kırılma anı, takımının şampiyonluğa yol...",idiom,Yol açmak,Zero-shot
2,Komşuluk festivali düzenleme görevini üstlenen...,idiom,Kolları sıvamak,Zero-shot
3,Şirketteki başarısızlıkla ilgili yapılan yorum...,idiom,Üstüne almak,Zero-shot
4,Sorunları erkenden çözerek kriz anlarında zama...,idiom,Zaman kazanmak,Zero-shot


In [16]:
df_eval.head()

,submission,category,idiom,type
0,Yeni öğrendiği şarkıyı sokak köşesinde söyleye...,idiom,İçi erimek,Zero-shot
1,Bir lider olarak çalışanlarının hatalarını def...,idiom,Defterden silmek,Zero-shot
2,"Yönetici, iş yerindeki moral ve motivasyonu yü...",idiom,Nabzını tutmak,Zero-shot
3,Şirketteki gizli toplantıyı kazara sızdırınca ...,idiom,Ayvayı yemek,Zero-shot
4,"Film eleştirmenleri, yönetmenin eserindeki sim...",idiom,Üzerinde durmak,Zero-shot


In [17]:
df_test.head()

,idiom,submission,category,rating,likes,dislikes,reports,idiom_indices,idiom_words,lemmas,words
0,adım atmak,"Sen bu adama adım mı attın ki, o sana yakınlaş...",idiom,0.75,3,0,1,"[3, 5]","['adım', 'attın']","['se', 'bu', 'ada', 'adım', 'mı', 'atmak', 'ki...","['Sen', 'bu', 'adama', 'adım', 'mı', 'attın', ..."
1,adım atmak,Bütün zorluklara rağmen yeni bir işe adım attı.,idiom,1.00,3,0,0,"[6, 7]","['adım', 'attı']","['bütün', 'zor', 'rağmen', 'yen', 'bir', 'işem...","['Bütün', 'zorluklara', 'rağmen', 'yeni', 'bir..."
2,adım atmak,Adım attığı bu hayatın başına ne gibi olaylar ...,idiom,0.75,3,0,1,"[0, 1]","['Adım', 'attığı']","['adım', 'atmak', 'bu', 'hayat', 'baş', 'ne', ...","['Adım', 'attığı', 'bu', 'hayatın', 'başına', ..."
3,adım atmak,"Bölüm başkanı, akademik dünyaya adım atan genç...",idiom,0.50,1,0,1,"[5, 6]","['adım', 'atan']","['bölüm', 'başka', ',', 'akademik', 'Dünya', '...","['Bölüm', 'başkanı', ',', 'akademik', 'dünyaya..."
4,adım atmak,Yeni bir deneyime adım atıyorum.,idiom,0.75,3,0,1,"[3, 4]","['adım', 'atıyorum']","['yen', 'bir', 'deneyim', 'adım', 'atmak', '.']","['Yeni', 'bir', 'deneyime', 'adım', 'atıyorum'..."


In [18]:
# Encode the 'category' column
label_encoder = LabelEncoder()
df_train['category_encoded'] = label_encoder.fit_transform(df_train['category'])
df_eval['category_encoded'] = label_encoder.transform(df_eval['category'])
df_test['category_encoded'] = label_encoder.transform(df_test['category'])

In [19]:
df_train

,submission,category,idiom,type,category_encoded
0,Eşitlik için yapılan mücadelelere karşı çıkmak...,idiom,Karşı çıkmak,Zero-shot,0
1,"Oyunun kırılma anı, takımının şampiyonluğa yol...",idiom,Yol açmak,Zero-shot,0
2,Komşuluk festivali düzenleme görevini üstlenen...,idiom,Kolları sıvamak,Zero-shot,0
3,Şirketteki başarısızlıkla ilgili yapılan yorum...,idiom,Üstüne almak,Zero-shot,0
4,Sorunları erkenden çözerek kriz anlarında zama...,idiom,Zaman kazanmak,Zero-shot,0
...,...,...,...,...,...
3451,Kütüphanede çalışmak için sessiz bir yer almış...,nonidiom,Yer almak,Zero-shot,1
3452,"Sprinter, yarışın son metresinde zamanı iyi ku...",nonidiom,Zaman kazanmak,Zero-shot,1
3453,Sergi alanının ortasına bir sanat eseri koydu ...,nonidiom,Ortaya koymak,Zero-shot,1
3454,Eski battaniyeyi köpeğin üstünden atmak için n...,nonidiom,Üstünden atmak,Zero-shot,1


In [20]:
df_eval

,submission,category,idiom,type,category_encoded
0,Yeni öğrendiği şarkıyı sokak köşesinde söyleye...,idiom,İçi erimek,Zero-shot,0
1,Bir lider olarak çalışanlarının hatalarını def...,idiom,Defterden silmek,Zero-shot,0
2,"Yönetici, iş yerindeki moral ve motivasyonu yü...",idiom,Nabzını tutmak,Zero-shot,0
3,Şirketteki gizli toplantıyı kazara sızdırınca ...,idiom,Ayvayı yemek,Zero-shot,0
4,"Film eleştirmenleri, yönetmenin eserindeki sim...",idiom,Üzerinde durmak,Zero-shot,0
...,...,...,...,...,...
859,"Koşu parkurunda, atletler yorulan arkadaşların...",nonidiom,Başa geçmek,Zero-shot,1
860,"Kitapçı, yeni gelen kitapları rafa koymadan ön...",nonidiom,El koymak,Zero-shot,1
861,"Çocuklar, köpeklerini sıkı bir şekilde tutarak...",nonidiom,Sıkı durmak,Zero-shot,1
862,"Tiyatro oyuncuları, sahnenin üstüne çıkmak içi...",nonidiom,Üste çıkmak,Zero-shot,1


In [21]:
df_test

,idiom,submission,category,rating,likes,dislikes,reports,idiom_indices,idiom_words,lemmas,words,category_encoded
0,adım atmak,"Sen bu adama adım mı attın ki, o sana yakınlaş...",idiom,0.75,3,0,1,"[3, 5]","['adım', 'attın']","['se', 'bu', 'ada', 'adım', 'mı', 'atmak', 'ki...","['Sen', 'bu', 'adama', 'adım', 'mı', 'attın', ...",0
1,adım atmak,Bütün zorluklara rağmen yeni bir işe adım attı.,idiom,1.00,3,0,0,"[6, 7]","['adım', 'attı']","['bütün', 'zor', 'rağmen', 'yen', 'bir', 'işem...","['Bütün', 'zorluklara', 'rağmen', 'yeni', 'bir...",0
2,adım atmak,Adım attığı bu hayatın başına ne gibi olaylar ...,idiom,0.75,3,0,1,"[0, 1]","['Adım', 'attığı']","['adım', 'atmak', 'bu', 'hayat', 'baş', 'ne', ...","['Adım', 'attığı', 'bu', 'hayatın', 'başına', ...",0
3,adım atmak,"Bölüm başkanı, akademik dünyaya adım atan genç...",idiom,0.50,1,0,1,"[5, 6]","['adım', 'atan']","['bölüm', 'başka', ',', 'akademik', 'Dünya', '...","['Bölüm', 'başkanı', ',', 'akademik', 'dünyaya...",0
4,adım atmak,Yeni bir deneyime adım atıyorum.,idiom,0.75,3,0,1,"[3, 4]","['adım', 'atıyorum']","['yen', 'bir', 'deneyim', 'adım', 'atmak', '.']","['Yeni', 'bir', 'deneyime', 'adım', 'atıyorum'...",0
...,...,...,...,...,...,...,...,...,...,...,...,...
2079,üzerinde durmak,Bu ipin üzerinde duramıyorum.,nonidiom,1.00,5,0,0,"[2, 3]","['üzerinde', 'duramıyorum']","['bu', 'ip', 'Üzer', 'durmak', '.']","['Bu', 'ipin', 'üzerinde', 'duramıyorum', '.']",1
2080,üzerinde durmak,İpin üzerinde tek ayağıyla duruyordu.,nonidiom,1.00,5,0,0,"[1, 4]","['üzerinde', 'duruyordu']","['ip', 'Üzer', 'tek', 'Ayağ', 'durmak', '.']","['İpin', 'üzerinde', 'tek', 'ayağıyla', 'duruy...",1
2081,üzerinde durmak,Bilgisayarın üzerinde kitapları duruyor,nonidiom,1.00,4,0,0,"[1, 3]","['üzerinde', 'duruyor']","['bilgisayar', 'Üzer', 'kitap', 'durmak']","['Bilgisayarın', 'üzerinde', 'kitapları', 'dur...",1
2082,üzerinde durmak,Üzerinde çilekler duran tartı bana uzattı.,nonidiom,1.00,1,0,0,"[0, 2]","['Üzerinde', 'duran']","['Üzer', 'Çilek', 'durmak', 'tartı', 'ban', 'u...","['Üzerinde', 'çilekler', 'duran', 'tartı', 'ba...",1


In [22]:
class TextDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)


In [23]:
# Initialize tokenizer
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

# Tokenize the data
train_encodings = tokenizer(list(df_train['submission']), truncation=True, padding=True, return_tensors="pt")
val_encodings = tokenizer(list(df_eval['submission']), truncation=True, padding=True, return_tensors="pt")
test_encodings = tokenizer(list(df_test['submission']), truncation=True, padding=True, return_tensors="pt")
# Convert to torch tensors
train_labels = torch.tensor(df_train['category_encoded'].values)
val_labels = torch.tensor(df_eval['category_encoded'].values)
test_labels = torch.tensor(df_test['category_encoded'].values)

# Prepare to datasets
train_dataset = TextDataset(train_encodings, train_labels)
val_dataset = TextDataset(val_encodings, val_labels)
test_dataset = TextDataset(test_encodings, test_labels)

# DataLoader
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
num_labels = len(label_encoder.classes_)
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=num_labels)

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
optimizer = AdamW(model.parameters(), lr=5e-6)

epochs = 4
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for batch in tqdm(train_loader):
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        total_loss += loss.item()

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

    # Average loss across all batches
    avg_train_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch + 1}/{epochs} | Train Loss: {avg_train_loss}")

    # Evaluation step
    model.eval()
    total_eval_accuracy = 0
    for batch in tqdm(val_loader):
        batch = {k: v.to(device) for k, v in batch.items()}
        with torch.no_grad():
            outputs = model(**batch)

        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)
        accuracy = (predictions == batch['labels']).cpu().numpy().mean() * 100
        total_eval_accuracy += accuracy

    avg_val_accuracy = total_eval_accuracy / len(val_loader)
    print(f"Epoch {epoch + 1}/{epochs} | Validation Accuracy: {avg_val_accuracy:.2f}%")


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
  0%|          | 0/432 [00:00<?, ?it/s]<ipython-input-22-845b27163e62>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings

In [ ]:
# Ensure the model is in evaluation mode
model.eval()

# Move the model to the appropriate device
model.to(device)

# Initialize variables to track accuracy
total_correct = 0
total_predictions = 0

# Store true labels and predictions for F1 score calculation
all_labels = []
all_predictions = []

# No gradient updates needed for evaluation
with torch.no_grad():
    for batch in test_loader:
        # Move batch to the same device as the model
        batch = {k: v.to(device) for k, v in batch.items()}

        # Forward pass
        outputs = model(**batch)

        # Get predictions
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)

        # Update tracking variables
        total_correct += (predictions == batch['labels']).sum().item()
        total_predictions += predictions.size(0)

        # Store predictions and true labels
        all_predictions.extend(predictions.cpu().numpy())
        all_labels.extend(batch['labels'].cpu().numpy())

# Calculate accuracy
test_accuracy = total_correct / total_predictions
print(f'Test Accuracy: {test_accuracy:.4f}')

# Calculate F1 scores
f1_weighted = f1_score(all_labels, all_predictions, average='weighted')
f1_macro = f1_score(all_labels, all_predictions, average='macro')

print(f'Weighted F1 Score: {f1_weighted:.4f}')
print(f'Macro F1 Score: {f1_macro:.4f}')


In [ ]:
# Generate classification report
report = classification_report(all_labels, all_predictions, digits=4)
print(f'Classification Report for {modelname}, language {language.upper()}, Train {input.upper()}, Test {test.upper()}:')
print(report)